In [801]:
#import libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains

import pymysql
import configparser
import mysql.connector

print('Libraries Imported.')

Libraries Imported.


In [802]:
# retruns a new web driver object
def createNewDriver():   

    options = webdriver.ChromeOptions() 
    # options.add_argument('--headless') # run driver without browser
    driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe', 
                                options=options)

    return driver

In [803]:
# navigate driver to proper frame within Product Line tab
def navigateToProductList(driver):
    url = 'https://www.echo-usa.com/Support-Help/Parts-Lookup'

    driver.get(url)
    # navigate to product line tab
    frameset = driver.find_element_by_tag_name('frameset')

    frame = frameset.find_element_by_tag_name('frame')
    frame_name = frame.get_attribute('name')

    # grab last 6 chars of frame name since it changes everytime page is loaded
    frame_suffix = frame_name[11:17]
    # print(frame_suffix)

    # set driver in frame that contains necessary info
    driver.switch_to.frame(frame)

    # find product line tab
    product_line = driver.find_element_by_id("Tab2")

    # select product line tab
    product_line.click()
    # print(browser.page_source)

    # set names/IDs of frames based on 6 chars that were previously saved
    prodLineFrameID = 'iframeSearchProdLine_' + frame_suffix
    detailFrameID = 'plDetailFrame'
    dataFrameName = 'ARICAT6TREE_' + frame_suffix

    # navigate into frame that contains jstree of all products/parts
    frame = driver.find_element_by_id(prodLineFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_id(detailFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_name(dataFrameName)
    driver.switch_to.frame(frame)

    # click into ECHO jstree
    cur_node = driver.find_element_by_id('pn18')
    cur_node.click()

    return frame_suffix

In [804]:
MANUFACTURER = 'Echo'


tool_id = 0
attachment_id = 0
diagram_id = 0
attachment_diagram_id = 0

def getNodes(unorderedList, is_attachment):

    global tool_id
    global attachment_id
    global diagram_id
    global attachment_diagram_id
    global myCursor
    
    is_attachment_next = is_attachment

    # get all list items from unordered list param
    nodes = unorderedList.find_elements_by_tag_name("li")

    # iterate through all list items in current unordered list
    for node in nodes:      
        # grab and store node ID
        node_id = node.get_attribute('id')

        # Skip Attachments and Accessories
        if node_id == "assemTree-item-.18.19":
            continue
        
        # wait until element is present
        wait = WebDriverWait(driver, 10)
        node = wait.until(EC.presence_of_element_located((By.ID, node_id)))

        # grab and store ID of corresponding anchor
        anchor_id = node_id + '_anchor'

        # make driver wait for anchor to be present before clicking
        wait = WebDriverWait(driver, 10)
        # anchor = wait.until(EC.element_to_be_clickable((By.ID, anchor_id)))
        anchor = wait.until(EC.presence_of_element_located((By.ID, anchor_id)))
        anchor.click()
        
        text = anchor.text
       
        model = None

        ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)    
        wait = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions)
        node = wait.until(EC.presence_of_element_located((By.ID, node_id)))
        # node = driver.find_element_by_id(node_id)

        if node.get_attribute('data-assembly-type') =="AT":
            is_attachment_next = True
        elif node.get_attribute('data-assembly-type') == "MD":
            serialNumIndex = text.find('S/N')
            if (serialNumIndex != -1):
                sn_range = text[serialNumIndex + 5:]
                model = text[0 : serialNumIndex -1]
            else:
                model = text
                sn_range = None

            myCursor = tool_db.cursor()

            if is_attachment is False:
                tool_id += 1
                sql = ('INSERT INTO tools (tool_id, manufacturer, model, serial_number_range) VALUES (%s, %s, %s, %s)')
                vals = (tool_id, MANUFACTURER, model, sn_range)

            else:
                attachment_id += 1
                sql = ('INSERT INTO attachments (attachment_id, manufacturer, model, tool_id) VALUES (%s, %s, %s, %s)')
                vals = (attachment_id, MANUFACTURER, model, tool_id)

            myCursor.execute(sql, vals)
            tool_db.commit()


        # (base case) check if node is a parent, if false gather info from leaf node
        # need to check thumb-url in case there is a false leaf (item is leaf node but contains no components)
        elif node.get_attribute('data-assembly-type') == "IP": #('data-assembly-is-parent') == "false" and node.get_attribute('data-assembly-thumb-url') is not None:
            # print('Reached leaf node.')
            if text == 'No Diagrams Required':
                continue
            else:
                if is_attachment is False:
                    diagram_id += 1
                    sql = ('INSERT INTO diagrams (diagram_id, manufacturer, description, tool_id) VALUES (%s, %s, %s, %s)')
                    vals = (diagram_id, MANUFACTURER, text, tool_id)
                else:
                    attachment_diagram_id += 1
                    sql = ('INSERT INTO attachment_diagrams (diagram_id, manufacturer, description, attachment_id) VALUES (%s, %s, %s, %s)')
                    vals = (attachment_diagram_id, MANUFACTURER, text, attachment_id)
                
                myCursor.execute(sql, vals)
                tool_db.commit()
                
                getComponents(is_attachment, diagram_id, attachment_diagram_id)
                continue
            
        # else, click the element to expand child list
        #else:
        # get xpath of current node
        xpath = '//li[@id = \'' + node_id + '\' and @aria-expanded = \'true\' and @aria-busy = \'false\']'

        # wait for list item to be visible
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))

        # once visible get a new reference to it, as the old reference is most likely stale
        node = driver.find_element_by_xpath(xpath) 

        # find next unordered list element and call getParts recursively to iterate through next level
        getNodes(node.find_element_by_tag_name("ul"), is_attachment_next)#, tool_id, attachment_id, diagram_id)

In [805]:
def getComponents(is_attachment, diagram_id, attachment_diagram_id):
    driver.switch_to.parent_frame()
    
    frame = driver.find_element_by_name('plDetail')

    driver.switch_to.frame(frame)
    driver.switch_to.frame(2)

    table = driver.find_element_by_tag_name('table')

    manufacturer = 'Echo'
    sku = None
    description = None

    for row in table.find_elements_by_tag_name('tr'):
        index = 0
        for cell in row.find_elements_by_tag_name('td'):
            if index == 3:
                sku = cell.text
            elif index == 4:
                description = cell.text
            index += 1
        if is_attachment is False:
            sql = ('INSERT INTO components (manufacturer, sku, description, diagram_id) VALUES (%s, %s, %s, %s)')
            vals = (manufacturer, sku, description, diagram_id)
        else:
            sql = ('INSERT INTO attachment_components (manufacturer, sku, description, diagram_id) VALUES (%s, %s, %s, %s)')
            vals = (manufacturer, sku, description, attachment_diagram_id) 
            
        myCursor.execute(sql, vals)
        tool_db.commit()

    returnToProductList()


In [806]:
def returnToProductList():
    driver.switch_to.parent_frame()
    driver.switch_to.parent_frame()

    frameName = 'ARICAT6TREE_' + frame_suffix

    # navigate into frame that contains product list
    frame = driver.find_element_by_name(frameName)
    driver.switch_to.frame(frame)


In [807]:
def connectMySQL():
    HOST = "ifixit-hiring.cxbmajqbo6pd.us-west-1.rds.amazonaws.com"
    USERNAME = "admin"
    PASSWORD = "mq76CKehrqTY7hgQZDRU"
    DATABASE = "tool_scraping"

    tool_db = mysql.connector.connect(
        host = HOST,
        user = USERNAME,
        password = PASSWORD,
        database = DATABASE
    )

    return tool_db

In [808]:
# create new webdriver to begin exploring target data
driver = createNewDriver()
frame_suffix = navigateToProductList(driver)

tool_db = connectMySQL()

# select first ul that contains all subdirectories
productList = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//ul[contains(@class, \'jstree-container-ul\')][li[@aria-level = \'1\']]')))

# pass list of products into getNodes(), a recursive func
getNodes(productList, False)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=95.0.4638.54)
